<a href="https://colab.research.google.com/github/iraidaantropova/RS/blob/main/HW2_RS_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 33.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631.0,1.0,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Задание 1. Weighted Random Recommendation
Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

Можно сэмплировать товары случайно, но пропорционально какому-либо весу
Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
#Задание
#def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
        Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    #return recs.tolist()

In [8]:
#Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,7,"[865569, 886703, 889731, 893400, 995436, 10205..."
1,14,"[949616, 1129609]"


In [9]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)

    return recs.tolist()

In [10]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 94.3 ms, sys: 489 µs, total: 94.8 ms
Wall time: 98.6 ms


,user_id,actual,random_recommendation
0,7,"[865569, 886703, 889731, 893400, 995436, 10205...","[6703800, 891148, 914557, 935126, 832298]"
1,14,"[949616, 1129609]","[874733, 947441, 1001728, 992292, 956370]"


In [11]:
#функция для получения весов товаров в зависимости от объёма продаж в денежном эквиваленте
def get_items_weights(df):
    total_sales=df['sales_value'].sum()
    items_weights=df.groupby('item_id').agg({'sales_value':'sum'}).reset_index().rename(columns={'sales_value':'weight'})
    items_weights['weight']=items_weights['weight'].apply(lambda x: x/total_sales)
    return items_weights

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    items = np.array(items_weights['item_id'])
    weights=np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False) #используем параметр p метода choice в который передадим веса товаров


    return recs.tolist()

In [13]:
%%time

# your_code
items_weights=get_items_weights(data_train)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 150 ms


,user_id,actual,random_recommendation,weighted_random_recommendation
0,7,"[865569, 886703, 889731, 893400, 995436, 10205...","[6703800, 891148, 914557, 935126, 832298]","[936897, 1003188, 939247, 1054999, 5566716]"
1,14,"[949616, 1129609]","[874733, 947441, 1001728, 992292, 956370]","[1096458, 858420, 1037840, 1049708, 911156]"


Задание 2. Расчет метрик

Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [14]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [15]:
def precision_at_k(result_recomendation, actual_recomendation, k=5):
    top_k = result_recomendation[:k]
    print(top_k)
    num_correct = len(set(top_k).intersection(actual_recomendation))
    print(num_correct)
    print(set(top_k))
    print(set(actual_recomendation))
    precision = num_correct / k
    return precision

In [16]:
# Запускаю функцию на одной строке.
precision_at_k(result.iloc[0]['random_recommendation'], result.iloc[0]['actual'])

[5586
4
{'8', '6', '[', '5'}
{'\n', '[', '6', ']', '9', '1', '2', ' ', '4', '5', '7', '8', '3', '0'}


0.8

In [17]:
result.iloc[0]['actual']

'[  821867   834484   856942   865456   889248   907957   914190   943316\n   951954   954486   958046   962568   969231   971585   979707   986947\n   990656   995242  1004906  1005186  1042083  1050310  1060819  1062002\n  1064441  1069733  1074612  1082185  1131115  1132771  6534544 13876341\n 15971874 17178953   883616   917704   931860   961554  1002032  1031190\n  8090541  8293439  9297615  9527329 15926712  1049998   861272   869465\n   877373   908213   933913   940947   945809   959316   978974  1031697\n  1041796  1048918  1081189  1101422  1115576  1122428  1132231  1132814\n  5577022  8091601  9296986  9677939 10356149 13417048 15741823 15830875]'

In [18]:
result['actual'] = result['actual'].apply(lambda x: [int(el) for el in x.replace('[', '').replace(']', '').split()])

In [19]:
def precision_at_k(result_recomendation, actual_recomendation, k=5):
    # get top-k recommendations from result_recomendation
    top_k = result_recomendation[:k]
    # calculate the number of correct recommendations
    num_correct = len(set(top_k).intersection(actual_recomendation))
    # calculate precision@k
    precision = num_correct / k
    return precision

In [20]:
result.apply(lambda row: precision_at_k(eval(row['random_recommendation']), row['actual']), axis=1).mean()

0.0005876591576885408